In [1]:
import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
from qiskit import Aer
from sklearn.decomposition import PCA
from tqdm.notebook import tqdm

import sys
sys.path.insert(0, '../../src/')
from neuralnetwork import *
from analysis import *
from utils import *

#%matplotlib notebook
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
backend = Aer.get_backend('qasm_simulator')

# Reguralized Feature Map

In [ ]:
np.random.seed(42)

n = 10
x = np.linspace(0, 1, n)
x = generate_meshgrid([x, x, x])
x = scaler(x, a=0, b=np.pi)

y = 0.5*np.ones((x.shape[0], 1))

### Single Circuit, Constant Function

In [ ]:
qnn = sequential_qnn(q_bits = [3],
                     dim = [3, 1],
                     reps = 1,
                     backend=backend,
                     shots=10000,
                     lr = 0.1)

In [ ]:
qnn.train(x, y, verbose=True)

In [ ]:
plt.plot(qnn.loss)

In [ ]:
a = 0.1
b = 0.9

x = np.vstack([a*np.ones(10), b*np.ones(10), np.linspace(0, np.pi, 10)]).T
print(x)

In [ ]:
y_pred = qnn.predict(x)

In [ ]:
plt.plot(x[:,2], y_pred)
plt.show()

In [ ]:
print(qnn.weight)

### Regularization

In [6]:
n = 4
x = np.linspace(0, 1, n)
x = generate_meshgrid([x, x, x])
x = scaler(x, a=0, b=np.pi)

y = scaler(x[:,1], a=0.1, b=0.9).reshape(-1,1)

In [7]:
np.random.seed(42)
model = RegularizedModel(n_features=3, n_targets=1, reps=1, backend=backend, shots=10000)
optimizer = Adam()
optimizer.initialize(model.theta.shape)

In [8]:
for i in range(100):
    y_pred = model.predict(x)
    loss = np.mean((y_pred - y)**2)
    print(loss)
    print(model.theta[:3])
    
    gradient = [model.gradient(x, y)]
    gradient = optimizer(gradient)[0]
    
    penalty = np.zeros_like(gradient)
    penalty[:3] = model.theta[:3]
    
    model.theta -= 0.1*gradient + 0.001*penalty
    

0.10069070321180557
[3.14159265 3.14159265 3.14159265]
0.09448316003472222
[3.23844975 3.03845169 3.03845493]
0.08990896821180556
[3.33413344 2.93567838 3.07801548]
0.08870896581597222
[3.42535509 2.83506432 3.11393206]
0.08893456112847223
[3.51220784 2.73787968 3.14063978]
0.0889485193576389
[3.59975626 2.64685286 3.16314648]
0.08890189930555556
[3.68657451 2.56196222 3.18867318]
0.08911723232638889
[3.75917482 2.48547201 3.21773309]
0.0893117954513889
[3.80236274 2.4167443  3.23853894]
0.0893716145138889
[3.82108584 2.35319267 3.25812812]
0.08871824571180556
[3.82473396 2.29378473 3.26949288]
0.08799748133680556
[3.82571155 2.23931739 3.27675359]
0.08833277607638888
[3.82812076 2.19041331 3.28969042]
0.08883524940972222
[3.83813143 2.1437432  3.306196  ]
0.08793369295138889
[3.85147545 2.1012443  3.32002983]
0.08854161503472223
[3.86446155 2.06364719 3.32884395]
0.08849093477430556
[3.87651511 2.03071757 3.33257905]
0.08897608399305557
[3.88847753 1.99908838 3.33424539]
0.08693602430

KeyboardInterrupt: 